In [50]:
# Import Required Libraries
import pandas as pd
import numpy as np
from faker import Faker

fake = Faker()

In [51]:
# Create the Billing Table
FactTable = pd.read_csv("data/FactTable.csv", parse_dates=['dimDateServicePK'], dayfirst=True)
DimHospital = pd.read_csv("data/DimHospital.csv")
Billing = pd.merge(FactTable, DimHospital, on='dimLocationPK', how='left')

# Apply Transformations
Billing["PaymentRecieved"] = Billing["Patient_Payment"] + Billing["AR"]
Billing["DiagnosisReportRef"] = "DIAG-" + Billing["FactTablePK"].astype(str)
Billing["ServiceReportRef"] = "SERV-" + Billing["FactTablePK"].astype(str)

column_mapping = {
    "FactTablePK" : "BillNbr",
    "dimDatePostPK" : "BillDate",
    "dimPatientPK" : "PatientNbr",
    "dimPhysicianPK" : "ProviderNbr",
    "LocationName" : "Facility",
    "Gross Expenses" : "BillAmount",
    "Insurance_Payment" : "InsuranceCoverage"
}

# Rename Columns
Billing = Billing.rename(columns=column_mapping)

# Select Columns
Billing = Billing[["BillNbr", "BillDate", "PatientNbr", "ProviderNbr", "Facility", "DiagnosisReportRef", "ServiceReportRef", "BillAmount", "InsuranceCoverage", "PaymentRecieved"]]

# Write the new Data to CSV File
Billing.to_csv('Spaghetti/Billing.csv', index=False)

C:\Users\murta\AppData\Local\Temp\ipykernel_7220\536781932.py:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  FactTable = pd.read_csv("data/FactTable.csv", parse_dates=['dimDateServicePK'], dayfirst=True)


In [52]:
# Create the Patient Table
DimPatient = pd.read_csv("data/DimPatient.csv")
MinServiceDate = FactTable.groupby("dimPatientPK").agg({"dimDateServicePK": "min"}).reset_index()
Patient = pd.merge(DimPatient, MinServiceDate, on='dimPatientPK', how='left')

# Apply Transformations
Patient.rename(columns={"Year of Birth": "year", "Month": "month", "Day": "day"}, inplace=True)
Patient["DOB"] = pd.to_datetime(Patient[["year", "month", "day"]])
Patient["RegistrationDate"] = Patient["dimDateServicePK"] - pd.to_timedelta(np.round(np.random.beta(0.5, 5, size=len(Patient)) * 10).astype(int), unit='days')
Patient["Address1"] = Patient.apply(lambda _: fake.secondary_address(), axis=1)
Patient["Address2"] = Patient.apply(lambda _: fake.building_number() + ' ' + fake.street_name(), axis=1)

column_mapping = {
    "dimPatientPK" : "PatientNbr",
    "PatientNumber" : "HealthCardNbr",
    "PatientGender" : "Gender",
    "Zip Codes" : "Zip"
}

# Rename Columns
Patient = Patient.rename(columns=column_mapping)

# Select Columns
Patient = Patient[["PatientNbr", "RegistrationDate", "HealthCardNbr", "FirstName", "LastName", "Email", "Gender", "DOB", "Address1", "Address2", "City", "State", "Zip"]]

# Write the new Data to CSV File
Patient.to_csv('Spaghetti/Patient.csv', index=False)

In [53]:
# Create the PatientSurvey Table
DimPatient = pd.read_csv("data/DimPatient.csv")
PatientSurvey = pd.merge(DimPatient, Patient, left_on='dimPatientPK', right_on='PatientNbr', how='left')

# Apply Transformations
PatientSurvey["SurveyDate"] = PatientSurvey["RegistrationDate"] + pd.to_timedelta(np.round(np.random.beta(1, 5, size=len(PatientSurvey)) * 3).astype(int), unit='days')
PatientSurvey["HeightIn"] = np.round(PatientSurvey["PatientHeight(in cms)"] / 2.54).astype(int)
PatientSurvey["WeightLbs"] = np.round(np.random.normal(150, 20, size=len(PatientSurvey)), 1)
PatientSurvey["WeightKgs"] = np.round(PatientSurvey["WeightLbs"] * 0.453592, 1)

column_mapping = {
    "PatientHeight(in cms)" : "HeightCms",
    "Tobacco" : "TobaccoUser",
    "Alcohol" : "AlcoholUser",
    "Exercise" : "ExerciseFrequency",
    "Diet" : "OnDiet"
}

# Rename Columns
PatientSurvey = PatientSurvey.rename(columns=column_mapping)

# Select Columns
PatientSurvey = PatientSurvey[["HealthCardNbr", "SurveyDate", "HeightCms", "HeightIn", "WeightLbs", "WeightKgs", "BloodGroup", "TobaccoUser", "AlcoholUser", "ExerciseFrequency", "OnDiet"]]

# Write the new Data to CSV File
PatientSurvey.to_csv('Spaghetti/PatientSurvey.csv', index=False)

In [54]:
# Create the Provider Table
DimPhysician = pd.read_csv("data/DimPhysician.csv")
DimSpeciality = pd.read_csv("data/DimSpeciality.csv")
Provider = pd.merge(DimPhysician, DimSpeciality, on='SpecialityCode', how='left')

# Apply Transformations
Provider["FirstName"] = Provider.apply(lambda _: fake.first_name(), axis=1)
Provider["Prefix"] = Provider["ProviderName"].apply(lambda name: 'Dr.' if 'Dr.' in name else '')
Provider["LastName"] = Provider["ProviderName"].apply(lambda name: name.replace('Dr. ', '') if 'Dr.' in name else name)
Provider["Email"] = Provider["FirstName"].str.lower() + '.' + Provider["LastName"].str.lower() + '@datacourse.com'

column_mapping = {
    "dimPhysicianPK" : "ProviderNbr",
    "ProviderNpi" : "NpiNbr",
    "ProviderFTE" : "FTE",
    "ProviderSpecialty" : "ProviderCategory"
}

# Rename Columns
Provider = Provider.rename(columns=column_mapping)

# Select Columns
Provider = Provider[["ProviderNbr", "NpiNbr", "FirstName", "LastName", "Email", "Prefix", "FTE", "ProviderCategory", "SpecialityCode"]]

# Write the new Data to CSV File
Provider.to_csv('Spaghetti/Provider.csv', index=False)

In [55]:
# Create the ProviderSpecialty Table
ProviderSpecialty = DimSpeciality.copy()

column_mapping = {
    "ProviderSpecialty" : "ProviderCategory",
    "SpecialityDesc" : "SpecialityDescription"
}

# Rename Columns
ProviderSpecialty = ProviderSpecialty.rename(columns=column_mapping)

# Select Columns
ProviderSpecialty = ProviderSpecialty[["SpecialityCode", "ProviderCategory", "SpecialityType", "SpecialityDescription"]]

# Write the new Data to CSV File
ProviderSpecialty.to_csv('Spaghetti/ProviderSpecialty.csv', index=False)

In [58]:
# Create the ClinicLocations Table
DimHospital = pd.read_csv("data/DimHospital.csv")
ClinicLocations = DimHospital.copy()

# Apply Transformations
ClinicLocations["Address1"] = ClinicLocations.apply(lambda _: fake.company() + ' Building', axis=1)
ClinicLocations["Address2"] = ClinicLocations.apply(lambda _: fake.building_number() + ' ' + fake.street_name(), axis=1)

column_mapping = {
    "dimLocationPK" : "ClinicNbr",
    "LocationName" : "DisplayName"
}

# Rename Columns
ClinicLocations = ClinicLocations.rename(columns=column_mapping)

# Select Columns
ClinicLocations = ClinicLocations[["ClinicNbr", "DisplayName", "Address1", "Address2", "City", "State", "Zip"]]

# Write the new Data to CSV File
ClinicLocations.to_csv('Spaghetti/ClinicLocations.csv', index=False)

,ClinicNbr,DisplayName,Address1,Address2,City,State,Zip
0,785522,Evergreen Clinic,"Mitchell, Moody and Clarke Building",643 Dustin Knoll,Columbus,Ohio,43109
1,785623,Twin Mountains Hospital,Shepherd and Sons Building,6883 Henry Stream,Jacksonville,Florida,32224
2,785724,Big Heart Community Hospital,"Lamb, Christensen and Peterson Building",72587 James Lane,Frankfort,Illinois,40601
3,785825,Pioneer Clinic,Payne Group Building,44470 Davis Cliffs,Auburn,New York,36849
4,785926,Fairmont Hospital Center,Wilson-Snow Building,659 Gonzalez Underpass,Beaumont,Texas,77708
5,786027,Angelstone Community Hospital,"Mcdowell, Sanchez and Williams Building",77733 Joseph Well,Austin,Texas,78749
6,786128,Genesis Hospital Center,Smith-Brewer Building,982 Adams Meadows,Marysville,Washington,98271
7,786229,Principal Medical Clinic,Jones Inc Building,1880 Mills Trail,Ann Arbor,Michigan,48103
8,786330,Fairview General Hospital,"Roberts, Nelson and White Building",627 Santos Station,Dublin,California,94568
9,786431,Guardian Medical Clinic,Reyes Inc Building,4761 Shirley Causeway,Bridgeton,New Jersey,63044
